In [1]:
import utils
from argparse import Namespace

args = Namespace()
args.generate_model="D:\\DDA4210\\facebookopt-1.3b"
args.ppl_model="D:\\DDA4210\\gpt"
args.prompts_name="lfqa.json"
args.use_gpu=True
args.prompt_max_length = None
args.max_new_tokens=200
args.min_new_tokens=50
args.gamma=0.25
args.delta=2.0
args.detection_z_threshold=4.0
args.generation_seed=42
args.normalizers=""
args.ignore_repeated_ngrams=False

model, tokenizer, device, pplmodel, ppltokenizer = utils.load_model(args)

input_text = utils.load_prompts()
print('prompt: ' + input_text)

gpu success
prompt: Does elevation change affect birds the same way it affects humans? (Or any other land animal for that matter)


In [2]:
without_wm, with_wm= utils.generate(input_text, 
                                        args, 
                                        model=model, 
                                        device=device, 
                                        tokenizer=tokenizer)
print('#######################################')
print('generated without watermark: ' + without_wm)
print('#######################################')
print('generated with watermark: ' + with_wm)
paraphrasing_wm = utils.paraphrasing_attack(with_wm)
print('#######################################')
print('paraphrasing with watermark: ' + paraphrasing_wm)
substitution_wm = utils.substitution_attack(with_wm)
print('#######################################')
print('substitution with watermark: ' + substitution_wm)

#######################################
generated without watermark:   I mean, you'd think that if it's a lot more difficult to fly around in the air, it's harder to eat.
I think you'd have to look it up, but I believe that there is a general consensus that it is.   I have never seen it mentioned as a negative, but I have also never seen any negative studies on the subject (I work with birds regularly).
#######################################
generated with watermark: 
Yes, there is some evidence that the heat from land animals contributes to climatic changes (I think it is more to do with humidity changes due to changes in air pressure).
Are there any accounts of changes in what land animals eat that have been published? Have there been changes to what land animals eat due to changes in elevation?
I think there have been some reports that have suggested that heat from land animals contributes to changes in the composition of food stocks (I think that is due to changes in water availab

In [6]:
without_wm_detection = utils.detect(without_wm, 
                                    args, 
                                    device=device, 
                                    model = model,
                                    tokenizer=tokenizer)
with_wm_detection = utils.detect(with_wm, 
                                args, 
                                device=device, 
                                model = model,
                                tokenizer=tokenizer)
rewritten_with_wm_detection = utils.detect(paraphrasing_wm, 
                                        args, 
                                        device=device, 
                                        model = model,
                                        tokenizer=tokenizer)
print('detect finished')

print('#######################################')
print('watermark words:' , with_wm_detection[4][1])

detect finished
#######################################
watermark words: ['Yes', 'there', 'is', 'some', 'evidence', 'that', 'heat', 'land', 'contributes', 'to', 'clim', 'changes', '(I', 'think', 'is', 'to', 'humidity', 'changes', 'due', 'to', 'changes', 'air', 'Are', 'there', 'accounts', 'changes', 'what', 'land', 'that', 'have?', 'Have', 'there', 'changes', 'to', 'what', 'land', 'due', 'to', 'changes?', 'I', 'think', 'there', 'have', 'some', 'reports', 'that', 'have', 'suggested', 'that', 'heat', 'land', 'contributes', 'to', 'changes', 'composition', 'stocks', '(I', 'think', 'that', 'is', 'due', 'to', 'changes', 'due', 'to', 'changes', 'air),', 'there', 'is', 'some', 'debate', 'about', 'whether', 'there', 'is', 'connection', 'changes', 'changes', 'to', 'land', 'diets', 'I', 'remember', 'seeing', 'that', 'research', 'that', 'showed', 'that', 'heat', 'land', 'is', 'what', 'impacts', 'composition', 'land', 'especially', 'for', 'that', 'live', 'at', 'elev', '(I', 'think', 'for', 'snow', '

In [7]:
ppl_without_wm = utils.compute_ppl(without_wm, 
                                    args,
                                    model=pplmodel,
                                    device=device, 
                                    tokenizer=ppltokenizer)
ppl_with_wm = utils.compute_ppl(with_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)
ppl_rewritten_with_wm = utils.compute_ppl(paraphrasing_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)
print('compute perplexity finished')

compute perplexity finished


In [8]:
analysis = {}
analysis['gamma'] = args.gamma
analysis['delta'] = args.delta
analysis['z_threshold'] = args.detection_z_threshold

analysis['T_with_watermark'] = with_wm_detection[0][1]
analysis['z_with_watermark'] = with_wm_detection[2][1]
analysis['p_with_watermark'] = with_wm_detection[3][1]
analysis['prediction_with_watermark'] = with_wm_detection[6][1]
# analysis['confidence_with_watermark'] = with_wm_detection[7][1]
analysis['ppl_with_watermark'] = ppl_with_wm

analysis['T_without_watermark'] = without_wm_detection[0][1]
analysis['z_without_watermark'] = without_wm_detection[2][1]
analysis['p_without_watermark'] = without_wm_detection[3][1]
analysis['prediction_without_watermark'] = without_wm_detection[6][1]
analysis['ppl_without_watermark'] = ppl_without_wm

analysis['T_attack'] = rewritten_with_wm_detection[0][1]
analysis['z_attack'] = rewritten_with_wm_detection[2][1]
analysis['p_attack'] = rewritten_with_wm_detection[3][1]
analysis['prediction_attack'] = rewritten_with_wm_detection[6][1]
analysis['ppl_attack'] = ppl_rewritten_with_wm

print(analysis)

{'gamma': 0.25, 'delta': 2.0, 'z_threshold': 4.0, 'T_with_watermark': '138', 'z_with_watermark': '14.4', 'p_with_watermark': '3.97e-47', 'prediction_with_watermark': 'True', 'ppl_with_watermark': tensor(4.4327), 'T_without_watermark': '25', 'z_without_watermark': '1.08', 'p_without_watermark': '0.141', 'prediction_without_watermark': 'False', 'ppl_without_watermark': tensor(0.5306), 'T_attack': '5', 'z_attack': '0.745', 'p_attack': '0.228', 'prediction_attack': 'False', 'ppl_attack': tensor(1.3963)}
